In [70]:
# from piec.drivers.utilities import PiecManager
from piec.drivers.keysight81150a import Keysight81150a
from piec.drivers.keysightdsox3024a import Dsox3024a
import pandas as pd
import time
# pm = PiecManager()

In [2]:
# pm.parse_resources()

In [3]:
awg = Keysight81150a("GPIB0::8::INSTR")
scope = Dsox3024a("GPIB0::7::INSTR")

In [4]:
awg.idn()

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [5]:
scope.idn()

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [65]:
#Setup functions
#Set up the pulse params
awg.configure_wf(channel='1', func="PULS", voltage="2.0", offset="1.0", frequency="1.0e7", duty_cycle="50")
#Sets the trigger source to manual
awg.configure_trigger(trigger_source="MAN")
#turns the output on
awg.output_enable(channel='1')

In [102]:
#Configure the scope
channel=1

scope.initialize()
scope.configure_timebase(time_base_type='MAIN', reference='CENTer', time_scale=f'{10e-9}', position=f'{0}') #this should be made general
scope.configure_channel(channel=f'{channel}', voltage_scale=f'{1}', impedance='FIFT', voltage_offset=3.0)#set both to 50ohm
scope.configure_trigger_characteristics(trigger_source='CHAN1', low_voltage_level='0.75', high_voltage_level='0.95', sweep='NORM')
scope.configure_trigger_edge(trigger_source='CHAN1', input_coupling='DC', level=1.0)
scope.initiate()
time.sleep(1)

In [103]:
awg.send_software_trigger()

In [104]:
#Capture Waveform
scope.setup_wf()
_, trace_t, trace_v  = scope.query_wf()#change
data = pd.DataFrame({"time (s)":trace_t, "voltage (V)": trace_v}) # Retrieve the data from the oscilloscope
data.head()

,time (s),voltage (V)
0,-5.000000e-08,-0.055276
1,-4.984375e-08,-0.055276
2,-4.968750e-08,-0.055276
3,-4.953125e-08,-0.055276
4,-4.937500e-08,-0.055276


In [105]:
data.to_csv(r"C:\Users\adqua\Documents\Lab\data\ultrafast_raw_out\10ns_delay.csv", index=False)